In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold
#https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py#L65

In [59]:
#Variable definitions...
n_splits = 5 
data = pd.DataFrame({'W':['EA-01','EA-02','EA-03','EA-04','EA-05'],'D':[1000,1000.5,1001,1001.5,1002],'G':[47.1,45.3,45.7,67.6,87.5]})
data

,W,D,G
0,EA-01,1000.0,47.1
1,EA-02,1000.5,45.3
2,EA-03,1001.0,45.7
3,EA-04,1001.5,67.6
4,EA-05,1002.0,87.5


In [66]:
#Rough Work...
np.argsort([1,0,2,1])

array([0, 1, 3, 2], dtype=int64)

In [64]:
def gkf_split(n_splits,groups):
    unique_grps, groups = np.unique(groups,return_inverse=True)
    num_groups = len(unique_grps)
    cv = []
    
    if(n_splits>num_groups):
        raise ValueError("Error..")
    
    n_samples_per_group = np.bincount(groups)
    indices = np.argsort(n_samples_per_group)[::-1] #Reverse of the indices..
    print(indices)
    n_samples_per_group = n_samples_per_group[indices] #values sorted in desc..
    
    n_samples_per_fold = np.zeros(n_splits) #Create 0's array of specified length.
    group_to_fold = np.zeros(num_groups) #Similarly here..
    
    for it, weight in enumerate(n_samples_per_group):
        lightest_fold = np.argmin(n_samples_per_fold)
        n_samples_per_fold[lightest_fold]+=weight
        group_to_fold[indices[it]] = lightest_fold
    
    which_grp_belong_to_which_fold = group_to_fold[groups]
    indices_train_test_split = np.arange(len(groups))
    
    for i in range(n_splits):
        test_indices = np.where(which_grp_belong_to_which_fold==i)[0]
        test_mask = np.zeros(len(groups), dtype=np.bool)
        test_mask[test_indices] = True
        train_indices = indices_train_test_split[np.logical_not(test_mask)]
        cv.append((train_indices,test_indices))
    return(cv)

In [44]:
class cv:
    def __init__(self,n_splits):
        self.n_splits = n_splits
        if(self.n_splits<=1):
            raise ValueError("Error..")
    def split(self,groups):
        train_test_splits_well = gkf_split(self.n_splits,groups)
        return(train_test_splits_well)

In [73]:
cv_obj = cv(6)
wn = data['W']
groups = [0,1,2,3,4]
print(cv_obj.split(groups))

ValueError: Error..

In [57]:
group_kfold = GroupKFold(n_splits=3)
for train_index, test_index in group_kfold.split(data.iloc[:,:-1], data['Gamma'], [0,1,2,3,4]):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 2 3] TEST: [1 4]
TRAIN: [1 2 4] TEST: [0 3]
TRAIN: [0 1 3 4] TEST: [2]


In [ ]:
"""
Pipeline Resource Useful links (In order):
https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/pipeline.py
https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/metaestimators.py
https://scikit-learn.org/stable/modules/compose.html#pipeline
https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/utils/validation.py#L164
https://scikit-learn.org/stable/modules/model_persistence.html
"""